In [1]:
import pandas as pd
import numpy as np

#-----------------------------------------------#

import time
import os
from dotenv import load_dotenv

#-----------------------------------------------#

from spotipy.oauth2 import SpotifyOAuth
import spotipy
import spotipy.util as util

#-----------------------------------------------#

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel

In [2]:
load_dotenv()

True

In [3]:
cid = '6ce665b3a2284af28da6d106169eae79'
secret = 'e97fd245c1ea4c3893690c6ba7164103'
redirect_uri = 'http://localhost:8080'
username = '31yxuvapn3lfzdhax4es457pyutq'

scope = 'user-top-read playlist-modify-public playlist-modify-private'
auth_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope, username=username)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
top20_raw = sp.current_user_top_tracks(time_range='short_term', limit=20)

In [5]:
for i, item in enumerate(top20_raw['items']):
    print(i+1, item['name'], '//', item['artists'][0]['name'])

1 Si No Estás // iñigo quintero
2 QLONA // KAROL G
3 Fire On Fire // Sam Smith
4 MI EX TENÍA RAZÓN // KAROL G
5 Tamagotchi // GUNTTER
6 LALA // Myke Towers
7 Sin Tiempo Para Bailar // iñigo quintero
8 greedy // Tate McRae
9 Hey Mor // Ozuna
10 AMARGURA // KAROL G
11 In The Stars // Benson Boone
12 PERRO NEGRO // Bad Bunny
13 Según Quién // Maluma
14 WANDA // Quevedo
15 Columbia // Quevedo
16 Sobredosis // iñigo quintero
17 Déjala Que Vuelva (feat. Manuel Turizo) // Piso 21
18 BESO // ROSALÍA
19 QLONA // KAROL G
20 Hasta Que Dios Diga // Anuel AA


In [90]:
tracks = top20_raw['items']
track_ids = []
track_names = []
features = []

for track in tracks:
    track_id = track['id']
    track_name = track['name']
    audio_features = sp.audio_features(track_id)
    
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])

In [91]:
top20_df = pd.DataFrame(features,index = track_names)


In [93]:
top20 = top20_df[['id', 'danceability',	'energy',	'key',	'loudness',	'mode',	'speechiness',	'acousticness',	'instrumentalness',	'liveness',	'valence',	'tempo', 'duration_ms']]

In [94]:
top20.head(5)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
Si No Estás,2HafqoJbgXdtjwCOvNEF14,0.537,0.421,5,-8.720,1,0.0285,0.8270,0.000000,0.1380,0.524,98.224,184061
QLONA,5RqSsdzTNPX1uzkmlHCFvK,0.841,0.738,7,-7.455,0,0.3070,0.5200,0.000000,0.0892,0.484,169.918,172798
Fire On Fire,7t3Xdbufg7q2onVsR8RBdY,0.584,0.408,1,-7.356,0,0.0461,0.4760,0.000000,0.1800,0.333,115.129,246735
MI EX TENÍA RAZÓN,54zcJnb3tp9c5OVKREZ1Is,0.634,0.725,5,-4.945,0,0.0587,0.0776,0.000000,0.2110,0.795,159.845,154375
Tamagotchi,5bfep56ypsHGIR4i5KPKJN,0.870,0.664,1,-8.716,1,0.0744,0.2330,0.000705,0.1410,0.840,98.013,151000


In [25]:
top20.shape

(20, 13)

In [26]:
ids_artists = []
print('|| Artists in my top 20: ||')
print('===========================')
for item in top20_raw['items']:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)

|| Artists in my top 20: ||
0jbo7KFNMiIkfBR6ih0yhm: iñigo quintero
790FomKkXshlbRYZFtlgla: KAROL G
2wY79sveU1sp5g7SokKOiI: Sam Smith
790FomKkXshlbRYZFtlgla: KAROL G
3psizJPIbIEEctInvdWSZk: GUNTTER
7iK8PXO48WeuP03g8YR51W: Myke Towers
0jbo7KFNMiIkfBR6ih0yhm: iñigo quintero
45dkTj5sMRSjrmBSBeiHym: Tate McRae
1i8SpTcr7yvPOmcqrbnVXY: Ozuna
790FomKkXshlbRYZFtlgla: KAROL G
22wbnEMDvgVIAGdFeek6ET: Benson Boone
4q3ewBCX7sLwd24euuV69X: Bad Bunny
1r4hJ1h58CWwUQe3MxPuau: Maluma
52iwsT98xCoGgiGntTiR7K: Quevedo
52iwsT98xCoGgiGntTiR7K: Quevedo
0jbo7KFNMiIkfBR6ih0yhm: iñigo quintero
4bw2Am3p9ji3mYsXNXtQcd: Piso 21
7ltDVBr6mKbRvohxheJ9h1: ROSALÍA
790FomKkXshlbRYZFtlgla: KAROL G
2R21vXR83lH98kGeO99Y66: Anuel AA


In [27]:
ids_artists = list(set(ids_artists))
print(f'Number of artists (without repetitions): {len(ids_artists)}')

Number of artists (without repetitions): 14


In [28]:
print('Artistas similares:')
print('=====================')
ids_similar_artists = []
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    for item in artists:
        artist_id = item['id']
        artist_name = item['name']
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id)

Artistas similares:
1GDbiv3spRmZ1XdM1jQbT7: NATTI NATASHA
4obzFoKoKRHIphyHzJ35G3: Becky G
0tmwSHipWxN12fsoLcFU3B: Manuel Turizo
5dbaLmK5SHLLg8Z4CcTJpX: Greeicy
1mcTU81TzQhprhouKaTkpq: Rauw Alejandro
07YUOmWljBTXwIseAUd9TW: Sebastian Yatra
1i8SpTcr7yvPOmcqrbnVXY: Ozuna
28gNT5KBp7IjEOQoevXf9N: Camilo
1r4hJ1h58CWwUQe3MxPuau: Maluma
1vyhD5VmyZ7KMfW5gqLgo5: J Balvin
2LRoIwlKmHjgvigdNGBHNo: Feid
6w3SkAHYPsQ1bxV7VDlG5y: Cazzu
4VMYDCV2IEDYJArk749S6m: Daddy Yankee
2R21vXR83lH98kGeO99Y66: Anuel AA
77ziqFxp5gaInVrF2lj4ht: Sech
329e4yvIujISKGKz1BZZbO: Farruko
4bw2Am3p9ji3mYsXNXtQcd: Piso 21
5lwmRuXgjX8xIwlnauTZIP: Romeo Santos
1SupJlEpv7RS2tPNRaHViT: Nicky Jam
47MpMsUfWtgyIIBEFOr4FE: Lunay
5TwUXL3I6RaLckHy8le2Hq: Thomas Day
5aHlyw2WWqZW0HLICBuv7U: Zach Hood
79AyR6ATpj2LTPxfb6FX50: Lauren Spencer Smith
7okSU80WTrn4LXlyXYbX3P: Clinton Kane
0znpFLuaey34oJTE1jHSnT: Alexander Stewart
6wlhqnATSJIc0NQ64VxOVJ: Matt Hansen
0sMAHzxguan2KpnKFZPW2d: Sadie Jean
0EiNdCUwM4B5GkTInLAyuj: Caleb Hearn
3IR6DvP0x2a6o

In [29]:
ids_artists.extend(ids_similar_artists)

In [30]:
ids_artists = list(set(ids_artists))
print(f'Number of artists (without repetitions): {len(ids_artists)}')

Number of artists (without repetitions): 187


In [31]:
new_releases = sp.new_releases(limit=20)['albums']

In [32]:
new_releases

{'href': 'https://api.spotify.com/v1/browse/new-releases?country=ES&offset=0&limit=20',
 'items': [{'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Mw40k757jZuiL0NIJpdO5'},
     'href': 'https://api.spotify.com/v1/artists/1Mw40k757jZuiL0NIJpdO5',
     'id': '1Mw40k757jZuiL0NIJpdO5',
     'name': 'GULEED',
     'type': 'artist',
     'uri': 'spotify:artist:1Mw40k757jZuiL0NIJpdO5'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4az97MtWmBQ5Db3GfDh9j9'},
     'href': 'https://api.spotify.com/v1/artists/4az97MtWmBQ5Db3GfDh9j9',
     'id': '4az97MtWmBQ5Db3GfDh9j9',
     'name': 'Morad',
     'type': 'artist',
     'uri': 'spotify:artist:4az97MtWmBQ5Db3GfDh9j9'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'H

In [33]:
print('')
print('Artists with new releases')
print('=====================')
for item in new_releases['items']:
    artist_id = item['artists'][0]['id']   #elegir solo el primer artista
    artist_name = item['artists'][0]['name']
    album_name = item['name']   # Nombre del album, puramente informativo en python
    release_date = item['release_date'] # Fecha de lanzamiento, puramente informativo en python
    print(f'{artist_id}: {artist_name} - // {album_name}, {release_date}')
    ids_artists.append(artist_id)


Artists with new releases
1Mw40k757jZuiL0NIJpdO5: GULEED - // Cuando Menos Lo Espera, 2023-07-07
6k8mwkKJKKjBILo7ypBspl: Ana Mena - // bellodrama, 2023-03-24
7iK8PXO48WeuP03g8YR51W: Myke Towers - // LA VIDA ES UNA, 2023-03-23
5XJDexmWFLWOkjOEjOVX3e: Eladio Carrion - // 3MEN2 KBRN, 2023-03-17
790FomKkXshlbRYZFtlgla: KAROL G - // MAÑANA SERÁ BONITO, 2023-02-24
2auC28zjQyVTsiZKNgPRGs: RM - // Indigo, 2022-12-02
6KImCVD70vtIoJWnq6nGn3: Harry Styles - // Harry's House, 2022-05-20
4q3ewBCX7sLwd24euuV69X: Bad Bunny - // Un Verano Sin Ti, 2022-05-06
2R21vXR83lH98kGeO99Y66: Anuel AA - // Las Leyendas Nunca Mueren, 2021-11-26
4dpARuHxo51G3z768sgnrY: Adele - // 30, 2021-11-19
6eUKZXaKkcviH0Ku9w2n3V: Ed Sheeran - // =, 2021-10-29
4gzpq5DPGxSnKTe4SA8HAU: Coldplay - // Music Of The Spheres, 2021-10-15
53KwLdlmrlCelAZMaLVZqU: James Blake - // Friends That Break Your Heart, 2021-10-08
4MzJMcHQBl9SIYSjwWn8QW: Spiritbox - // Eternal Blue, 2021-09-17
1vyhD5VmyZ7KMfW5gqLgo5: J Balvin - // JOSE, 2021-09-1

In [34]:
ids_artists = list(set(ids_artists))
print(f'Number of artists (without repetitions): {len(ids_artists)}')

Number of artists (without repetitions): 199


In [35]:
id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Processing artist {i+1} of {nartists}...')
    albums = sp.artist_albums(id_artist, limit=1)  # to avoid having a huge list
    for album in albums['items']:
        id_albums.append(album['id'])
    
    time.sleep(2)  # Adds 3-seconds delay

print('Done!')


Processing artist 1 of 199...
Processing artist 2 of 199...
Processing artist 3 of 199...


KeyboardInterrupt: 

In [36]:
id_albums

['2u2IVr3kVR13lr517rvcjM', '65LmFuLBa1yBX5yfuuOMh2', '4W1o9T3ghMa61HFLUAIOHj']

In [37]:
id_tracks = []
nalbums = len(id_albums)
for i, id_album in enumerate(id_albums):
    print(f'Processing album {i+1} of {nalbums}...')
    album_tracks = sp.album_tracks(id_album, limit=2)
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
    
    time.sleep(2)  # Adds 3-seconds delay

print(f'Done! Total number of pre-candidate tracks: {len(id_tracks)}')


Processing album 1 of 3...
Processing album 2 of 3...
Processing album 3 of 3...
Done! Total number of pre-candidate tracks: 6


In [38]:
id_tracks

['5XJWvtPzYIST3qfMiiFsQZ',
 '2lQl5Zgvqgqoa2EzwbIpVM',
 '1QBoyx9EOFbnFQJGCyEzwz',
 '3yh46UlxlTAFpTSaQI3XDm',
 '58ove9RPbXPbEjDdw1YNEk',
 '7EQ6caOA5TbIHtGi6k2mfF']

In [39]:
track_names = []
features = []
ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    print(f'Processing track {i+1} of {ntracks}...')
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    
    # Do not include tracks without "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])

    time.sleep(2)  # Adding some seconds delay

print('Done!')

candidates_df = pd.DataFrame(features, index = track_names)

Processing track 1 of 6...
Processing track 2 of 6...
Processing track 3 of 6...
Processing track 4 of 6...
Processing track 5 of 6...
Processing track 6 of 6...
Done!


implementing a simple caching mechanism that could help reduce the number of API calls
    
    track_cache = {}


    track_names = []
    features = []
    ntracks = len(id_tracks)

    for i, track_id in enumerate(id_tracks):
        print(f'Processing track {i+1} of {ntracks}...')

        # Check if the track's details are in the cache
        if track_id in track_cache:
            track_info = track_cache[track_id]
        else:
            track_info = sp.track(track_id)
            track_cache[track_id] = track_info  # Cache the data for future use
            time.sleep(3)  # Adding some seconds delay to avoid rate limit

        track_name = track_info['name']
        audio_features = sp.audio_features(track_id)

        # Check if the audio features are in the cache
        if track_id in track_cache and 'features' in track_cache[track_id]:
            features_info = track_cache[track_id]['features']
        else:
            features_info = audio_features[0] if audio_features and audio_features[0] is not None else None
            if features_info:
                track_cache[track_id]['features'] = features_info  # Cache the features

        if features_info:
            track_names.append(track_name)
            features.append(features_info)

    print('Done!')

    candidates_df = pd.DataFrame(features, index=track_names)


In [95]:
candidates_df.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Intro,0.728,0.711,8,-6.498,1,0.2420,0.1610,0.000003,0.137,0.422,101.198,audio_features,5XJWvtPzYIST3qfMiiFsQZ,spotify:track:5XJWvtPzYIST3qfMiiFsQZ,https://api.spotify.com/v1/tracks/5XJWvtPzYIST...,https://api.spotify.com/v1/audio-analysis/5XJW...,130571,4
Kittyponeando - interlude,0.433,0.971,8,-3.988,1,0.0936,0.0722,0.851000,0.298,0.962,199.829,audio_features,2lQl5Zgvqgqoa2EzwbIpVM,spotify:track:2lQl5Zgvqgqoa2EzwbIpVM,https://api.spotify.com/v1/tracks/2lQl5Zgvqgqo...,https://api.spotify.com/v1/audio-analysis/2lQl...,92406,4
Love Me For Another Day,0.536,0.423,6,-9.158,1,0.0314,0.6950,0.000185,0.117,0.228,134.027,audio_features,1QBoyx9EOFbnFQJGCyEzwz,spotify:track:1QBoyx9EOFbnFQJGCyEzwz,https://api.spotify.com/v1/tracks/1QBoyx9EOFbn...,https://api.spotify.com/v1/audio-analysis/1QBo...,183000,3
I Don't Wanna Leave Just Yet,0.413,0.405,7,-7.319,1,0.0311,0.5050,0.000000,0.110,0.216,178.158,audio_features,3yh46UlxlTAFpTSaQI3XDm,spotify:track:3yh46UlxlTAFpTSaQI3XDm,https://api.spotify.com/v1/tracks/3yh46UlxlTAF...,https://api.spotify.com/v1/audio-analysis/3yh4...,185280,4
Madrid,0.641,0.657,0,-6.031,1,0.0334,0.0482,0.000000,0.171,0.457,88.025,audio_features,58ove9RPbXPbEjDdw1YNEk,spotify:track:58ove9RPbXPbEjDdw1YNEk,https://api.spotify.com/v1/tracks/58ove9RPbXPb...,https://api.spotify.com/v1/audio-analysis/58ov...,228818,4
Venga Tía,0.753,0.639,0,-7.435,1,0.0949,0.0757,0.000000,0.382,0.787,110.030,audio_features,7EQ6caOA5TbIHtGi6k2mfF,spotify:track:7EQ6caOA5TbIHtGi6k2mfF,https://api.spotify.com/v1/tracks/7EQ6caOA5TbI...,https://api.spotify.com/v1/audio-analysis/7EQ6...,181091,4


In [41]:
candidates = candidates_df[['id', 'danceability',	'energy',	'key',	'loudness',	'mode',	'speechiness',	'acousticness',	'instrumentalness',	'liveness',	'valence',	'tempo', 'duration_ms']]


In [ ]:
candidates.to_csv('./spotify_data_cache/mi_base_de_datos.csv', index=False)

In [ ]:
candidates = pd.read_csv('./datos/mi_base_de_datos.csv')

In [64]:
top20_mtx = top20.iloc[:,1:].values
candidatos_mtx = candidates.iloc[:,1:].values

In [70]:
scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
can_scaled = scaler.fit_transform(candidatos_mtx)

In [72]:
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
can_norm = np.sqrt((can_scaled*can_scaled).sum(axis=1))

In [46]:
nt20 = t20_scaled.shape[0]
ncan = can_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
can = can_scaled/can_norm.reshape(ncan,1)

In [73]:
cos_sim = linear_kernel(t20,can)
cos_sim.shape

(20, 6)

In [74]:
print(cos_sim[6,5])
print(cos_sim[3,4])

0.06004984044163581
-0.22378708579860151


In [80]:
def obtener_candidatos(pos, cos_sim, ncands, umbral = 0.4):
    # Obtener todas las pistas candidatas por encima de umbral
    idx = np.where(cos_sim[pos,:]>=umbral)[0] # ejm. idx: [27, 82, 135]
    
    # Y organizarlas de forma descendente (por similitudes de mayor a menor)
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] porque por defecto argsort organiza de manera ascendente

    # Si hay más de "ncands", retornar máximo "ncands"
    if len(idx) >= ncands:
        cands = idx[0:ncands]
    else:
        cands = idx
  
    return cands

In [81]:
for i in range(5):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i}==> pistas filtradas: {cands}, similitudes: {cos_sim[i,cands]}')

0==> pistas filtradas: [2 3], similitudes: [0.79645576 0.60535444]
1==> pistas filtradas: [0], similitudes: [0.56965963]
2==> pistas filtradas: [2 3], similitudes: [0.47936509 0.44349952]
3==> pistas filtradas: [1], similitudes: [0.60631099]
4==> pistas filtradas: [5], similitudes: [0.48869137]


In [87]:
ids_t20 = []
ids_playlist = []

for i in range(top20.shape[0]):
    print(top20.index[i])   # Nombre de la pista en el top-20
    ids_t20.append(top20['id'][i])
    
    # Obtener listado de candidatos para esta pista
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    
    # Si hay pistas relacionadas obtener los ids correspondientes
    # e imprimir en pantalla
    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        # Obtener los ids correspondientes e imprimir en pantalla
        for j in cands:
            id_cand = candidates['id'][j]
            ids_playlist.append(id_cand)
            
            # E imprimir en pantalla el candidato
            print(f'   {candidates.index[j]}')

Si No Estás
     ***No se encontraron pistas relacionadas***
QLONA
     ***No se encontraron pistas relacionadas***
Fire On Fire
     ***No se encontraron pistas relacionadas***
MI EX TENÍA RAZÓN
     ***No se encontraron pistas relacionadas***
Tamagotchi
     ***No se encontraron pistas relacionadas***
LALA
     ***No se encontraron pistas relacionadas***
Sin Tiempo Para Bailar
     ***No se encontraron pistas relacionadas***
greedy
     ***No se encontraron pistas relacionadas***
Hey Mor
     ***No se encontraron pistas relacionadas***
AMARGURA
     ***No se encontraron pistas relacionadas***
In The Stars
     ***No se encontraron pistas relacionadas***
PERRO NEGRO
     ***No se encontraron pistas relacionadas***
Según Quién
     ***No se encontraron pistas relacionadas***
WANDA
     ***No se encontraron pistas relacionadas***
Columbia
     ***No se encontraron pistas relacionadas***
Sobredosis
   Love Me For Another Day
Déjala Que Vuelva (feat. Manuel Turizo)
     ***No se encontrar

C:\Users\Felix\AppData\Local\Temp\ipykernel_23168\722148203.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ids_t20.append(top20['id'][i])
C:\Users\Felix\AppData\Local\Temp\ipykernel_23168\722148203.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  id_cand = candidates['id'][j]


In [61]:
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]
ids_playlist_dep = list(set(ids_playlist_dep))

In [62]:
print(len(ids_playlist_dep))

1


In [ ]:
# Now try creating the playlist
pl = sp.user_playlist_create(user=username, 
                             name='Spotipy Recommender Playlist', 
                             description="Playlist created with the recommendation system")

sp.playlist_add_items(pl['id'],ids_playlist_dep)